# Get Data Liga Ingris
## Kelompok 15 Pemrograman Jaringan Komputer
- source: https://sport.detik.com/sepakbola/jadwal/liga-inggristhis 
- project: https://github.com/dirsulaiman/pemrogjarkom-kelompok15

### Library Required
* BeautifulSoup
* Pandas
* Psycopg2
* Getpass
* Twilio

## Get Data from Tag HTML

In [14]:
import urllib.request as urllib2
from bs4 import BeautifulSoup

site = "https://sport.detik.com/sepakbola/jadwal/liga-inggris";
page = urllib2.urlopen(site)
soup = BeautifulSoup(page, 'html.parser')

matchList = soup.findAll('div', attrs={'class':'match_schedule'})
#print(match)

In [15]:
# make list
matchList_table = []

#foreach match list
for m in matchList:
    
    #foreach li on matchday
    for li in m.findAll('li'):
        #make match dictionary
        match = {'matchday':'', 'is_pass':'', 'team1':'', 'team2':'', 'score_team1':'', 'score_team2':'', 'date':'', 'time':''}

        #get matchday name
        match['matchday'] = next(m.find('div', 'title').stripped_strings)
        
        #stripped string on li, return generator
        contents = li.stripped_strings
        
        #convert generator to list
        texts = list(contents)
       
        #check if li has class='full-time'
        if li.get('class'):
            #print(li.get('class')[0])
            match['is_pass'] = True
            match['team1'] = texts[0]
            match['team2'] = texts[1]
            match['score_team1'] = texts[2]
            match['score_team2'] = texts[3]
            match['date'] = texts[5]
            #add data to matchList_table[]
            matchList_table.append(match)
        else:
            match['is_pass'] = False
            match['team1'] = texts[0]
            match['team2'] = texts[1]
            match['date'] = texts[2]
            match['time'] = texts[3]
            #add data to matchList_table[]
            matchList_table.append(match)

#print(matchList_table)

## Display Data as Table

In [16]:
import pandas as panda
dataFrames = panda.DataFrame(matchList_table)

#display data as table
display(dataFrames)

,matchday,is_pass,team1,team2,score_team1,score_team2,date,time
0,Matchday 21 of 38,False,Arsenal FC,Manchester United FC,,,02/01/20,03:00
1,Matchday 21 of 38,False,West Ham United FC,AFC Bournemouth,,,02/01/20,00:30
2,Matchday 21 of 38,False,Norwich City FC,Crystal Palace FC,,,02/01/20,00:30
3,Matchday 21 of 38,False,Manchester City FC,Everton FC,,,02/01/20,00:30
4,Matchday 21 of 38,False,Watford FC,Wolverhampton Wanderers,,,01/01/20,22:00
...,...,...,...,...,...,...,...,...
95,Matchday 12 of 38,True,Newcastle United FC,AFC Bournemouth,2,1,09/11/19,
96,Matchday 12 of 38,True,Burnley FC,West Ham United FC,3,0,09/11/19,
97,Matchday 12 of 38,True,Chelsea FC,Crystal Palace FC,2,0,09/11/19,
98,Matchday 12 of 38,True,Norwich City FC,Watford FC,0,2,09/11/19,


## Save data to Database

In [1]:
import psycopg2 as pg
import getpass

#username = input('Username = ')
pas = getpass.getpass('Password = ')
conn = pg.connect(database="kelompok_15", user="dir", password=pas, host="127.0.0.1", port="5432")

Password = ········


In [ ]:
cur = conn.cursor()
for row in matchList_table:
    if row['is_pass']:
        cur.execute("insert into liga_ingris (matchday, is_pass, team1, team2, score_team1, score_team2, date) values ('" +row['matchday']+ "'," +str(row['is_pass'])+ ",'" +row['team1']+ "','" +row['team2']+ "','" +row['score_team1']+ "','" +row['score_team2']+ "','" +row['date']+ "')")        
    else:
        cur.execute("insert into liga_ingris (matchday, is_pass, team1, team2, date, time) values ('" +row['matchday']+ "'," +str(row['is_pass'])+ ",'" +row['team1']+ "','" +row['team2']+ "','" +row['date']+ "','" +row['time']+ "')")
conn.commit()
#conn.close()

## Defenition Functions

In [44]:
import datetime as dt
from datetime import datetime
from twilio.rest import Client
from time import sleep

#parameter date="dd/mm/yy" time="hh:mm"
def toDatetime(date, time):
    dat = date.split('/')
    tim = time.split(':')
    return dt.datetime(int("20"+dat[2]), int(dat[1]), int(dat[0]), int(tim[0]), int(tim[1]))

def timeToNotify(time):
    return time - dt.timedelta(hours=6)

# return delay in seconds with type float
def getDelay(time):
    return timeToNotify(time).timestamp()-datetime.now().timestamp()

#parameter message="string", to="+62xxx", delay=delays
def sendWhatsappMessage(message, to, delay):
    account_sid = 'AC380ba9e71406a71a7042377a46a5615e'
    auth_token = 'a66ff691ee115435f2203f2127c2c91f'
    client = Client(account_sid, auth_token)

    from_whatsapp_number = 'whatsapp:+14155238886'
    to_whatsapp_number = 'whatsapp:'+to
    sleep(delay)
    client.messages.create(body=message, from_=from_whatsapp_number, to=to_whatsapp_number)

## Multi Thread

In [45]:
from threading import Thread
from time import sleep

thread = Thread(target = sendWhatsappMessage, args = ("ini pesan","+6285825327542", 60))
thread.start()
print("thread finished...exiting")

thread finished...exiting


In [ ]:
#thread list for each match
threads_list = []
i = 0

#fetch all data from database
cur = conn.cursor()
cur.execute('select * from liga_ingris')
table_rows = cur.fetchall()
conn.commit()

for row in table_rows:
    if not row[2]:
        #get delay to notify via WA
        delay = getDelay((toDatetime(row[7], row[8])))
        if delay >= 0:
            print(delay)
            #make message
            messageContent = "*Informasi Akan Bertanding* \n"+row[3]+" *VS* "+row[4]+"\n"+row[1]+"\n"+"Pada "+row[7]+" Pukul "+row[8]
            thread = Thread(target = sendWhatsappMessage, args = (messageContent,"+6285825327542", delay))
            threads_list.append(thread)
            #start thread
            #each thread sleep for delay time given
            threads_list[i].start()
            threads_list[i].join()
            i = i+1
            #print(messageContent)
            
conn.close()

672.5078361034393
62998.01568007469
